In [ ]:
# turn this into app

In [1]:
%load_ext autoreload
%autoreload 2

import requests, bs4, re
import time, os
import json
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import numpy as np

# from scraper import betfair, neds


In [2]:
STAKE = 25
COMM = 0.06

In [7]:
silk = re.compile('\([\d]+\)')

In [8]:
def beteasy(driver):
    html_be = driver.page_source
    soup = bs4.BeautifulSoup(html_be, 'html.parser')

    lines = [x for x in soup.find_all('div', {'class': 'RunnerRow__runnerRow--D1PmI'})]
    spans = []
    for line in lines:
        spans.append([x.text for x in line.find_all('span') if x.text not in ['Market Mover', 'FAV', 'EW']])
    be_df = pd.DataFrame(spans)
    be_df = be_df.set_index(1)
    print(be_df.shape[1])
    n_cols = be_df.shape[1]
    if n_cols == 12:
        be_df = be_df[[be_df.columns[-4], be_df.columns[-2]]]
    else:
        be_df = be_df[[be_df.columns[-3], be_df.columns[-1]]]
    be_df.columns = ['back', 'place']
    be_df.back = be_df.back.astype(float)
    be_df.place = be_df.place.astype(float)
    
    return be_df

In [21]:
def betfair(driver):
    html_betfair = driver.page_source
    soup = bs4.BeautifulSoup(html_betfair)
    racing_table = soup.find_all('table')
    tar_table = np.argmax([len(x.find_all('tr')) for x in racing_table])
    
    
    rows = []
    for row in racing_table[tar_table].find_all('tr'):
        rows.append([x.text for x  in row.find_all('div')])

    bf_df = pd.DataFrame(rows).dropna()
    if bf_df.shape[1] == 16:
        bf_df['silk'] = bf_df[5].apply(lambda x: silk.findall(x)[0])
    else:
        bf_df['silk'] = bf_df[6].apply(lambda x: silk.findall(x)[0])
    bf_df = bf_df.set_index('silk')
    bf_df = bf_df[[bf_df.columns[-6], bf_df.columns[-3]]]
    bf_df.columns = ['back_bf', 'lay']
    bf_df.back_bf = bf_df.back_bf.apply(lambda x: x.split(' ')[0]).astype(float)
    bf_df.lay = bf_df.lay.apply(lambda x: x.split(' ')[0]).astype(float)

    
    return bf_df

In [3]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--window-size=1920x1080")

browser = webdriver.Chrome(options=chrome_options)
browser.set_window_size(1920, 1080)
browser2 = webdriver.Chrome(options=chrome_options)
browser2.set_window_size(1920, 1080)

In [4]:
nds = 'https://beteasy.com.au/racing-betting'
bf = 'https://www.betfair.com.au/exchange/plus/horse-racing/next'
browser.get(nds)
browser2.get(bf)

## Manually Navigate to desired race

In [24]:
def main(neds_df, df_bf):
    final_df = neds_df.join(df_bf, how='outer')
    final_df = final_df[['back', 'lay', 'back_bf', 'place']]
    for c in ['back', 'lay', 'back_bf']:
        final_df[c] = final_df[c].apply(float)

    final_df['EV_noback'] = (STAKE * final_df.back - STAKE) * 1/final_df.back_bf + (0.7 * STAKE - STAKE) * (1 / final_df.place - 1/final_df.back_bf) + -STAKE * (1 - 1/final_df.place)
    final_df['EV_half'] = (STAKE * final_df.back - STAKE - STAKE/2 * (final_df.lay-1)) * 1/final_df.back_bf + (0.7 * STAKE - STAKE + STAKE/2 * (1 - COMM)) * (1 / final_df.place - 1/final_df.back_bf) + (-STAKE + STAKE/2 * (1 - COMM)) * (1 - 1/final_df.place)
    final_df['BB'] = (final_df.back - 1) / (final_df.lay - COMM) * STAKE
    final_df['BB_liability'] = (final_df.lay - 1) * STAKE

    return final_df[final_df.lay != 0]#.sort_values('EV_half', ascending=False)

In [26]:
button = widgets.Button(description="Run/Refresh")
button_cl = widgets.Button(description="Close")
display(button)
display(button_cl)
out = widgets.Output()

@out.capture()
def on_button_clicked(b):
    # final_df = neds_df.merge(df_bf, how='outer', on='num')
#     neds_df = neds(browser)
    be_df = beteasy(browser)
    df_bf = betfair(browser2)
    df = main(be_df, df_bf)
    
    clear_output()
    display(df)
    display(df[df.EV_noback > 5])
    display(df[df.BB > 18])
    
    
def close(b):
    browser.close()
    browser2.close()

button.on_click(on_button_clicked)
button_cl.on_click(close)
out

Button(description='Run/Refresh', style=ButtonStyle())

Button(description='Close', style=ButtonStyle())

Output()